# Business Data Science | Fall 2019
## Problem Set 5
### Rutwick Bhawsar | Shivang Arya

In [43]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### Problem 1. Question 1

In [36]:
df=pd.read_csv("NYC_stop_and_frisk.dat",sep=" ")
df = df.iloc[4:,0:-4].copy()
df.reset_index(drop=True,inplace=True)
df.rename(columns={'stop': 'Past_Arrests','and': 'Total_Population', 'frisk': 'Stopped_Population','data' : 'Precinct', '(with' : "Ethnicity", 'noise' : 'Crime'}, inplace=True)

In [37]:
df

,Past_Arrests,Total_Population,Stopped_Population,Precinct,Ethnicity,Crime
0,75,1720,191,1,1,1
1,36,1720,57,1,1,2
2,74,1720,599,1,1,3
3,17,1720,133,1,1,4
4,37,1368,62,1,2,1
5,39,1368,27,1,2,2
6,23,1368,149,1,2,3
7,3,1368,57,1,2,4
8,26,23854,135,1,3,1
9,32,23854,16,1,3,2


### Problem 1. Question 2. 

In [6]:
# Converting the DataFrame to Numeric. It was originaly stored in String.
df = df.apply(pd.to_numeric)

#Getting Total Population among the 75 precincts.
totalpop = df.iloc[:,0].sum()

#Getting Ethnic Population among the 75 precincts.
totalblack = df.loc[df['Ethnicity']==1].sum()[0]
totalhispanic = df.loc[df['Ethnicity']==2].sum()[0]
totalwhite = df.loc[df['Ethnicity']==3].sum()[0]

#Getting the fractions
fractionpopblack = (totalblack/totalpop)*100
fractionpopwhite = (totalwhite/totalpop)*100
fractionpophispanic = (totalhispanic/totalpop)*100

print()
print("Fraction of Black Population corresponding to Total Population:", fractionpopblack)
print("Fraction of Hispanic Population corresponding to Total Population:", fractionpophispanic)
print("Fraction of White Population corresponding to Total Population:", fractionpopwhite)

#Getting total stops.
totalstops = df.iloc[:,1].sum()

#Getting total stop as per ethnicity.
stopsblack = df.loc[df['Ethnicity']==1].sum()[1]
stopshispanic = df.loc[df['Ethnicity']==2].sum()[1]
stopswhite = df.loc[df['Ethnicity']==3].sum()[1]

#Getting the fractions.
fractionstopsblack = (stopsblack/totalstops)*100
fractionstopswhite = (stopswhite/totalstops)*100
fractionstopshispanic = (stopshispanic/totalstops)*100

print("\nFraction of Black Stops corresponding to Total Stops:", fractionstopsblack)
print("Fraction of Hispanic Stops corresponding to Total Stops:", fractionstopshispanic)
print("Fraction of White Stops corresponding to Total Stops:", fractionstopswhite)
print()


Fraction of Black Population corresponding to Total Population: 27.546911368656584
Fraction of Hispanic Population corresponding to Total Population: 25.574140350701068
Fraction of White Population corresponding to Total Population: 46.87894828064235

Fraction of Black Stops corresponding to Total Stops: 53.14937494451232
Fraction of Hispanic Stops corresponding to Total Stops: 31.66412304102072
Fraction of White Stops corresponding to Total Stops: 15.18650201446696



In [44]:
df['Past_Arrests'].shape

(900,)

In [45]:
fam = Poisson()
ind = Independence()
model1 = GEE.from_formula("Stopped_Population ~ Past_Arrests", 'Ethnicity', df, cov_struct=ind, family=fam)
result1 = model1.fit()
print(result1.summary())

ValueError: Leading dimension of 'exog' should match length of 'endog'

https://s3.amazonaws.com/assets.datacamp.com/production/course_15356/slides/chapter3.pdf

1. This link has a way to do this but does not include how to handle an exposure variable
2. We can manually code (take log of) the Y var and use normal regression (3 times, once for every ethnicity since its a control var) but then again haven't considerd handling of the exposure variable.
3. https://www.statsmodels.org/devel/generated/statsmodels.genmod.generalized_linear_model.GLM.html has a way to handle exposure variable
4. I was looking at https://stackoverflow.com/questions/37941881/how-to-implement-poisson-regression


## Problem 2.

In [53]:
df2=pd.read_csv("nba_cc_fake_data.csv",index_col=0)
df2.head()

,Comp,Height,Points,Salary
0,9.0,76.0,27.0,0.0
1,7.0,78.0,39.0,0.0
2,9.0,76.0,39.0,0.0
3,9.0,74.0,39.0,0.0
4,9.0,74.0,26.0,0.0


### Problem 2. Task 1

Explaination:

### Problem 2. Task 2

In [54]:
#Dividing the Data into training and testing
X = df2.iloc[:,0:3]
y = df2.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Running Linear Regression on it.
reg = LinearRegression().fit(X, y)

#Predicting for testing data.
y_pred = reg.predict(X_test)

In [55]:
#Seeing how well our model performed. 
r2_score(y_test.values, y_pred)

0.19176262013019696

### Problem 2. Task 3

In [56]:
df_nba=df2.copy()
for i in df_nba.index:
    if(df_nba.loc[i,'Salary']>0):
        df_nba.at[i,'Salary']=1
df_nba.head(10)

,Comp,Height,Points,Salary
0,9.0,76.0,27.0,0.0
1,7.0,78.0,39.0,0.0
2,9.0,76.0,39.0,0.0
3,9.0,74.0,39.0,0.0
4,9.0,74.0,26.0,0.0
5,8.0,75.0,46.0,0.0
6,8.0,78.0,35.0,0.0
7,7.0,75.0,56.0,0.0
8,9.0,77.0,37.0,0.0
9,9.0,78.0,63.0,1.0


In [58]:
#Dividing the Data into training and testing
X = df_nba.iloc[:,0:3]
y = df_nba.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
lr=LogisticRegression()
lr.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

We can try a different classification method here^. Maybe a decision tree

In [60]:
y_pred = lr.predict(X_test)
print(accuracy_score(y_test.values, y_pred))
print(roc_auc_score(y_test.values, y_pred))

0.944
0.5758514068059764


In [62]:
df_sal=df2.loc[df2['Salary']>0]
df_sal.head(10)

,Comp,Height,Points,Salary
9,9.0,78.0,63.0,496315.0
27,10.0,76.0,52.0,741265.0
30,8.0,78.0,62.0,292281.0
115,10.0,77.0,63.0,782210.0
117,8.0,76.0,50.0,259365.0
136,9.0,76.0,50.0,407135.0
147,10.0,78.0,59.0,709341.0
148,10.0,80.0,49.0,839476.0
150,10.0,78.0,47.0,956352.0
163,9.0,80.0,40.0,542106.0


In [63]:
#Dividing the Data into training and testing
X = df_sal.iloc[:,0:3]
y = df_sal.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
r2_score(y_test.values, y_pred)

0.5415468455217292

### Problem 2. Task 4

In [73]:
reg.predict(np.array([9,78,46]).reshape(1, -1))[0]

486918.2272167178

We can try a non linear transformation here^